<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Exercise 1:</b> Dataset Chat</h2>
<br>

**Welcome back! This is the first exercise in the course, so see what you can do!**

This notebook serves as a hands-on exercise following the "main-lecture" notebook. The exercises in this series utilizes the same dataset to gradually help us enable more complex LLM interactions.

In the previous notebook, we implemented a basic multi-agent system to generate synthetic multi-turn conversations. While useful for generating artificial dialogues, this implementation lacks practicality for end-user applications.

### **Learning Objectives:**

**In this notebook, we will:**

- Build a simple user-facing chatbot that interacts with a dataset.
- Address the challenge of handling datasets too large for our model’s context window.
- Develop a summarization pipeline to preprocess data efficiently.

This dataset will remain in use throughout the course, so our first step is to enable a simple interactive chat with it.

<hr><br>

### **Part 1:** Setting Up A Workshop Assistant Chatbot

From the previous exercise, you can specify a simple chatbot that interacts with the user using a simple loop. Based on this, the following function establishes a simple chatbot loop where a user can interact with an AI agent. If no processing function (chain) is provided, it defaults to an unimplemented generator that outputs a placeholder message.

In [ ]:
from time import sleep

def not_implemented_gen(state):
    """A placeholder generator that informs users the chain is not yet implemented."""
    message = "Chain Not Implemented. Enter with no inputs or interrupt execution to exit."
    for letter in message:
        yield letter
        sleep(0.005)

def chat_with_chain(state={}, chain=not_implemented_gen):
    """
    Interactive chat function that processes user input through a specified chain.
    
    Parameters:
        state (dict): Maintains chat history and context.
        chain (callable): Function to generate responses based on the chat history.
    """
    assert isinstance(state, dict)
    state["messages"] = state.get("messages", [])
    while True:
        try:
            human_msg = input("\n[Human]:")
            if not human_msg.strip(): break
            agent_msg = ""
            state["messages"] += [("user", human_msg)]
            print(flush=True)
            print("[Agent]: ", end="", flush=True)
            for token in getattr(chain, "stream", chain)(state):
                agent_msg += token
                print(getattr(token, "content", token), end="", flush=True)
            state["messages"] += [("ai", agent_msg)]
        except KeyboardInterrupt:
            print("KeyboardInterrupt")
            break

## Initialize chat with the placeholder generator
chat_with_chain()

<br>

From this, we can define a conversational pipeline with an LLM, a prompt template, and a starting state. A prompt, llm, and an output parser are provided, so please combine them together into a chain for your `chat_with_chain` function:

In [ ]:
from langchain_nvidia import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from functools import partial

## Define an NVIDIA-backed LLM
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

## Define a structured prompt
sys_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant for NVIDIA Deep Learning Institute (DLI). "
     "Assist users with their workshop-related queries using the provided context. "
     "Do not reference the 'context' as 'context' explicitly."),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])

## Construct the processing pipeline
chat_chain = sys_prompt | llm | StrOutputParser()

## Initialize chatbot state
state = {
    "messages": [("ai", "Hello! I'm the NVIDIA DLI Chatbot! How can I help you?")],
    "context": "",  # Empty for now; will be updated later
}

## Wrap function to integrate AI response generation
chat = partial(chat_with_chain, chain=chat_chain)

## Start the chatbot with the AI pipeline
chat(state)

<hr><br>

### **Part 2:** Pulling In Some Context

For this course, we will start by using a small dataset of workshop catalog from the GTC 2025 conference. This includes a real selection of workshops which were each proposed independently and vary in detail. This should be reminiscent of an organically-accumulated datapool... partially because it is one. The data can be found in [`gtc-data-2025.csv`](./gtc-data-2025.csv), so let's go ahead and load it in as a list!

In [ ]:
import pandas as pd
import json

## Load dataset
filepath = "gtc-data-2025.csv"
df = pd.read_csv(filepath)

## Convert to JSON for structured processing
raw_entries = json.loads(df.to_json(orient="records"))

## Display the first few records
raw_entries[:4]

<br>

We can quickly process them into a more natural format, and then try to concatenate them together to create a viable "context string" for our model. Automation to create context is quite common in real workflows to augment LLMs, so no reason not to do it here.

In [ ]:
def stringify(entry, description_key='description'):
    """Formats workshop details into a human-readable string."""
    return (
        f"{entry.get('name')}\n"
        f"Presenters: {entry.get('instructors')}\n"
        f"Description: {entry.get(description_key)}"
    )

## Convert dataset entries to structured text
raw_blurbs = [
    f"[Session {i+1}]\n{stringify(entry)}" 
    for i, entry in enumerate(raw_entries)
]

## Construct full context string
raw_context = "The following workshops are slated to be presented at NVIDIA's GTC 2025 Conference:\n\n"
raw_context += "\n\n".join(raw_blurbs)

## Display context statistics
print(f"Full Context Length (characters): {len(raw_context)}")
print("-"*40)
print(raw_context[:2000])  # Preview the first portion

In [ ]:
## Using your previous abstraction, pass the context into your prompt and see if it works:
## TODO: Initialize your state based on your opinionated chat chain
state = {}

try:
    ## TODO: Perform the conversation with your long context (it's ok if it fails)
    pass
except Exception as e:
    print(e)

<br>

<details><summary><b>Hint</b></summary>

Recall that we just have a `chat` function which wraps a reusable chain. So we can just invoke it using `chat(state)` for some `state`. Then, we just need to figure out what should go in our prompt. To refresh your memory:

```python
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
     "Assist users with their course-related queries using the provided context. "
     "Do not reference the 'context' as 'context' explicitly."),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])

```

</details>
<details><summary><b>Solution</b></summary>

Given that our prompt expects to take in a dictionary which includes a `context` (interpretable as strings) and `messages` (interpretable as a list of messages like `[("user", "Hello World")]`), we can initiate our prompt with no message history and the `raw_context` as our context.

```python
state = {"messages": [], "context": raw_context,}
try:
    chat(state)
except Exception as e:
    print(e)

```

</details>
<br>

This probably didn't work, and for good reason. The model is launched with a max context of `2^13 = 8192` tokens, and the current context is probably a bit too long. Let's verify it with a [tokenizer](https://huggingface.co/unsloth/Meta-Llama-3.1-8B-Instruct/blob/main/tokenizer.json) for the model in question.

In [ ]:
from transformers import PreTrainedTokenizerFast

llama_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json", clean_up_tokenization_spaces=True)

def token_len(text):
    """Counts token length of given text."""
    return len(llama_tokenizer.encode(text=text))

print(f"String Length of Context: {len(raw_context)}")
print(f"Token Length of Context: {token_len(raw_context)}")

## Preview context
print(raw_context[:2000])

<br>

**You may be thinking "don't most models have much longer contexts," and you would be right with a few key caveats:**

- Using an API service, you would still be paying for the tokens anyway, so maybe having a long static context isn't the best idea?
- Even if your context length is supported, most models still experience some amount of quality degredation as your inputs get longer. Furthermore, more inputs = more opportunities for conflicting data and text structures.
- For an arbitrary document or even a document pool, you are likely to find yourself stretching into max context more often than not. Even if a model is good for this dataset, will it still work well for a database?

In our case, we are operating on a sample of course descriptions of various detail and quality, which adds up to a large pool of inconsistent entries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sorted_raw_blurbs = sorted(raw_blurbs, key=token_len)

def plot_token_len(entries, color="green", alpha=1, len_fn=token_len):
    """Plots token lengths of all entries."""
    plt.bar(x=range(len(entries)), height=[len_fn(v) for v in entries], width=1.0, color=color, alpha=alpha)

plot_token_len(sorted_raw_blurbs, color="green")
plt.xlabel("Entry Sample Number")
plt.ylabel("Token Length")
plt.show() 

print("SHORTEST ENTRIES:")
sample_blurbs = sorted_raw_blurbs[:3] + sorted_raw_blurbs[-3:]

for entry in sample_blurbs:
    print(entry, "\n")

<hr><br>

### **Part 3:** Summarizing Our Long Context

Maybe we can convert each of these entries into something shorter and more uniform? Maybe as a preprocessing step, we can just process all of these entries into a more consistent form. Not only will this help our model reason about the full context, but we'll also be able to leverage the entries' uniform nature to improve the consistency of our prompt.

In [ ]:
%%time
## TODO: Create a symmary system message to instruct the LLM.
## Reuse the chat_chain as-it-was, remembering that it expects "messages" and "context"
summary_msg = (
    "Summarize the presentation description down to only a few important sentences."
    " Start with '(Summary) '"
    ## Feel free to customize
)

def summarize(context_str, summary_msg=summary_msg):
    return "(Summary) No summary"

print(summarize(stringify(raw_entries[1])))

<br>

<details><summary><b>Solution</b></summary>

```python
return chat_chain.invoke({
    "messages": [("user", summary_msg)],
    "context": context_str
})
```

</details>
<br>

It's natural language and not required to be well-formatted, but we can sufficiently prompt-engineer it for a simple text-to-text transformation function. We can also use the LangChain batching primitives to greatly simplify our concurrency management:

In [ ]:
%%time
from langchain_core.runnables import RunnableLambda
from tqdm.auto import tqdm
import threading

batch_inputs = [stringify(entry_dict) for entry_dict in raw_entries]

## Simple version of a batched process. No progress bar
# summaries = RunnableLambda(summarize).batch(batch_inputs, config={"max_concurrency": 20})

## Modified version which also has progress bars! Marginally-slower, same backbone
def batch_process(fn, inputs, max_concurrency=20):
    lock = threading.Lock()
    pbar = tqdm(total=len(inputs))
    def process_doc(value):
        try:
            output = fn(value)
        except Exception as e: 
            print(f"Exception in thread: {e}")
        with lock:
            pbar.update(1)
        return output
    try:
        lc_runnable = fn if hasattr(fn, "batch") else RunnableLambda(process_doc)
        return lc_runnable.batch(inputs, config={"max_concurrency": max_concurrency})
    finally:
        pbar.close()

summaries = batch_process(summarize, batch_inputs)

In [ ]:
summaries[:5]

<br>

Now that we have this new summary, we can see what happens when we use this synthetic description instead of our original ones, and consider how our context length is reduced.

In [ ]:
#############################################################################
## Defined Earlier

# def stringify(entry, description_key='description'):
#     return (
#         f"{entry.get('name')}"
#         f"\nPresentors: {entry.get('instructors')}"
#         f"\nDescription: {entry.get(description_key)}"
#     )

## Defined Earlier
#############################################################################

for summary, pres_entry in zip(summaries, raw_entries):
    words = summary.split()
    ## Remove "summary" or "(summary)" from text
    if "summary" in words[0].lower():
        words = words[1:]
    pres_entry["summary"] = " ".join(words)

print(stringify(raw_entries[0], "summary"))
raw_entries[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

contexts_with_summaries = [stringify(entry, "summary") for entry in raw_entries]
contexts_with_descripts = [stringify(entry) for entry in raw_entries]

def plot_token_len(entries, color="green", alpha=1, len_fn=token_len):
    plt.bar(x=range(len(entries)), height=[len_fn(v) for v in entries], width=1.0, color=color, alpha=alpha)    

## Create arrays of the token lengths
sorted_summs = [v for _,v in sorted(zip((token_len(x) for x in contexts_with_summaries), contexts_with_summaries))]
sorted_origs = [v for _,v in sorted(zip((token_len(x) for x in contexts_with_descripts), contexts_with_descripts))]
aligned_summs = [v for _,v in sorted(zip((token_len(x) for x in contexts_with_descripts), contexts_with_summaries))]
plot_token_len(sorted_origs, alpha=0.6, color="green")
plot_token_len(sorted_summs, alpha=0.6, color="grey")
## Lightgreen bars represent the new context length for their respective original green bars
plot_token_len(aligned_summs, alpha=0.6, color="lightgreen")
plt.xlabel("Entry Sample")
plt.ylabel("Token Length")
plt.show() 

print("Samples:")
sorted_raw_entries = sorted(raw_entries, key=(lambda v: token_len(str(v.get("description")))))
for entry in sorted_raw_entries[:3] + sorted_raw_entries[-3:]:
    print(
        f"{entry.get('name')}"
        f"\nPresentors: {entry.get('instructors')}"
        f"\nDescription: {entry.get('description')}"
        f"\nSummary: {entry.get('summary')}\n\n"
    )

<br>

Sounds like a promising direction! Let's implement it in practice and apply this change over all of our entries.

In [ ]:
## Construct full context string
new_context = "The following workshops are slated to be presented at NVIDIA's GTC 2025 Conference:\n\n"
new_context += "\n\n".join(contexts_with_summaries)
print("New Context Length:", len(new_context))
print(f"New Context Tokens: {token_len(new_context)}")

## Preview context
print(new_context[:2000])

And all of a sudden, we're below our input size threshold! Time to just swap out our context and test it out.

In [ ]:
#############################################################################
## Defined Earlier. Feel free to play around with this

## Define an NVIDIA-backed LLM
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

## Define a structured prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful instructor assistant for NVIDIA Deep Learning Institute (DLI). "
     "Assist users with their course-related queries using the provided context. "
     "Do not reference the 'context' as 'context' explicitly."),
    ("user", "<context>\n{context}</context>"),
    ("ai", "Thank you. I will not restart the conversation and will abide by the context."),
    ("placeholder", "{messages}")
])

## Construct the processing pipeline
chat_chain = prompt | llm | StrOutputParser()

## Initialize chatbot state
state = {
    "messages": [("ai", "Hello! I'm the NVIDIA DLI Chatbot! How can I help you?")],
    "context": "",  # Empty for now; will be updated later
}

## Wrap function to integrate AI response generation
chat = partial(chat_with_chain, chain=chat_chain)

## Defined Earlier
#############################################################################

state = {
    "messages": [],
    "context": new_context,
}

try:  ## HINT: Consider putting your call logic in the try-catch
    chat(state)
except Exception as e:
    print(e)

In [ ]:
## Consider saving the material as well, since it will be useful for later
## For those who may take this course over multiple sessions, a version is provided.
with open("simple_long_context.txt", "w") as f:
    f.write(new_context)

<hr><br>

### **Part 4:** Reflecting On This Exercise

This exercise was pretty easy, and did actually lead to an interesting system in some sense. 
- On a superficial level, all we did was take an overlong context and convert it down to a not-too-overlong context.
- Using other terms, we "canonicalized" the elements in the global environment into a form that help make up a reasonable "canonical context" to our primary LLM.
- Pessimistically, we have created a very short-term solution to our limited-context-space multi-turn problem by making the context *a little shorter* than our maximum input length.
- Optimistically, we now have a reusable context which can help to keep our entire context within our model's input domain for most single-turn problems (including those that may complement a multi-turn solution).

We will continue to use the results of this exercise throughout the coming notebooks, so hopefully the utility of this simple process becomes apparent as we go along!

<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>